## Taxi Trajectory Prediction

Here are the links to papers and github repo with solution

https://arxiv.org/pdf/1508.00021.pdf  
https://github.com/adbrebs/taxi

In [1]:
! ls /home/ubuntu/taxi

arrival-clusters.pkl			    taxi1.ipynb   train.csv
arrivals.pkl				    test2pickle   val_idx.pkl
__MACOSX				    test.csv	  X_train.pkl
metaData_taxistandsID_name_GPSlocation.csv  tmp		  X_valid.pkl
models					    train2pickle


In [2]:
import torch

In [3]:
import pickle
from fastai.imports import *
from fastai.structured import *
from fastai.column_data import *
import torch.nn as nn
from fastai.dataset import *
from fastai.learner import *

In [4]:
path = "/home/ubuntu/taxi/"

In [47]:
# import zipfile
# zip_ref = zipfile.ZipFile('train.csv.zip', 'r')
# zip_ref.extractall(path)
# zip_ref.close()

In [48]:
train = pd.read_csv("train.csv")

In [49]:
test = pd.read_csv("test.csv")

In [50]:
train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [51]:
list(train.POLYLINE[:1])

['[[-8.618643,41.141412],[-8.618499,41.141376],[-8.620326,41.14251],[-8.622153,41.143815],[-8.623953,41.144373],[-8.62668,41.144778],[-8.627373,41.144697],[-8.630226,41.14521],[-8.632746,41.14692],[-8.631738,41.148225],[-8.629938,41.150385],[-8.62911,41.151213],[-8.629128,41.15124],[-8.628786,41.152203],[-8.628687,41.152374],[-8.628759,41.152518],[-8.630838,41.15268],[-8.632323,41.153022],[-8.631144,41.154489],[-8.630829,41.154507],[-8.630829,41.154516],[-8.630829,41.154498],[-8.630838,41.154489]]']

In [52]:
list(test.POLYLINE[:1])

['[[-8.585676,41.148522],[-8.585712,41.148639],[-8.585685,41.148855],[-8.58573,41.148927],[-8.585982,41.148963],[-8.586396,41.148954],[-8.586072,41.14872],[-8.586324,41.147847],[-8.586999,41.14746],[-8.586576,41.147154],[-8.584884,41.146623]]']

In [53]:
test.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,T1,B,NaN,15.0,20000542,1408039037,A,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[..."
1,T2,B,NaN,57.0,20000108,1408038611,A,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-..."
2,T3,B,NaN,15.0,20000370,1408038568,A,False,"[[-8.585739,41.148558],[-8.58573,41.148828],[-..."
3,T4,B,NaN,53.0,20000492,1408039090,A,False,"[[-8.613963,41.141169],[-8.614125,41.141124],[..."
4,T5,B,NaN,18.0,20000621,1408039177,A,False,"[[-8.619903,41.148036],[-8.619894,41.148036]]"


In [54]:
meta = pd.read_csv("metaData_taxistandsID_name_GPSlocation.csv")

In [55]:
meta.head()

,ID,Descricao,Latitude,Longitude
0,1,Agra,41.1771457135,-8.609670
1,2,Alameda,41.15618964,-8.591064
2,3,Aldoar,41.1705249231,-8.665876
3,4,Alfândega,41.1437639911,-8.621803
4,5,Amial,41.1835097223,-8.612726


In [56]:
meta.shape, test.shape, train.shape

((63, 4), (320, 9), (1710670, 9))

Ok. So we have 1.7 million training rides and only 320 test rides. Metadata is just having name of cities for lat/long. 

In [57]:
train.dtypes

TRIP_ID           int64
CALL_TYPE        object
ORIGIN_CALL     float64
ORIGIN_STAND    float64
TAXI_ID           int64
TIMESTAMP         int64
DAY_TYPE         object
MISSING_DATA       bool
POLYLINE         object
dtype: object

### Step 1. Feature extraction

Winner's process to convert csv to hdf5 and extract features. I would prefer working on `pandas` for the same as it will allow me to look at how data looks. Replicating all steps present in `csv_to_hdf5.py` to make pandas df

In [58]:
train2 = train.copy()
test2 = test.copy()
meta2 = meta.copy()

Below code proves that all the taxi ids in test set are also present in training set. We don't have any new taxi id in test data. But there are **20 origin calls** that are not in training data but are in test data. 

In [6]:
s=0
for l in test.TAXI_ID.unique():
    if l not in train.TAXI_ID.unique():
        s = s+1
p=0
g = []
for l in test.ORIGIN_CALL.unique():
    if l not in train.ORIGIN_CALL.unique():
        p = p+1
        g.append(l)
s,p

`g` as saved above has list of origin_calls that are in test but not in train

In [62]:
g[1:]

[85698.0,
 85224.0,
 86098.0,
 86436.0,
 74478.0,
 81753.0,
 67507.0,
 65127.0,
 66996.0,
 72185.0,
 75460.0,
 73870.0,
 72911.0,
 73071.0,
 75037.0,
 80148.0,
 66812.0,
 70885.0,
 76232.0]

Let's see how many unique `origin_calls` and `taxi_ids` do we have in both training and test set

In [63]:
len(train.TAXI_ID.unique()), len(test.TAXI_ID.unique()), len(train.ORIGIN_CALL.unique()), len(test.ORIGIN_CALL.unique())

(448, 244, 57106, 61)

So basically, approximately half of `taxi_ids` from training are there in test data.

Pre-calculated below on train set


In [5]:
lat_mean = 41.15731
lat_std = 0.074120656
long_mean = -8.6161413
long_std = 0.057200309

Making a new feature with only first 5 and last 5 coordinates. In cases where we don't have enough coordinates, we do `edge` padding (i.e. just adding first and last values in leftmost and rightmost sides respectively)

In [70]:
def start_stop_inputs(k, train, test):
    
    """
    Function used to take first k and last k coordinates from given polyline.
    For train - if total coordinates < 2 then return none, if > 2*k then return first k and last k
    and if > 2 but < 2*k then do edge padding and return.
    
    """
    result_train = []
    result_test = []
    
    for l in train[['LONGITUDE','LATITUDE']].iterrows():
        if len(l[1][0]) < 2 or len(l[1][1]) < 2:
            result_train.append(np.nan)
        elif len(l[1][0][:-1]) >= 2*k:
            result_train.append(np.concatenate([l[1][0][0:k],l[1][0][-k:],
                                          l[1][1][0:k],l[1][1][-k:]]).flatten())
        else:
            l1 = np.lib.pad(l[1][0][:-1], (0,4*k-len(l[1][0][:-1])), mode='edge')
            l2 = np.lib.pad(l[1][1][:-1], (0,4*k-len(l[1][1][:-1])), mode='edge')
            result_train.append(np.concatenate([l1[0:k],l1[-k:],l2[0:k],l2[-k:]]).flatten())

    for l in test[['LONGITUDE','LATITUDE']].iterrows(): 
        if len(l[1][0]) < 1 or len(l[1][1]) < 1:
            result_test.append(np.nan)
        elif len(l[1][0]) >= 2*k:
            result_test.append(np.concatenate([l[1][0][0:k],l[1][0][-k:],l[1][1][0:k],l[1][1][-k:]]).flatten())
        else:
            l1 = np.lib.pad(l[1][0], (0,4*k-len(l[1][0])), mode='edge')
            l2 = np.lib.pad(l[1][1], (0,4*k-len(l[1][1])), mode='edge')
            result_test.append(np.concatenate([l1[0:k],l1[-k:],l2[0:k],l2[-k:]]).flatten())
    
    return pd.Series(result_train), pd.Series(result_test)

In [75]:
import ast
def feature_ext(train, test):   
    """
    Function used to extract model features.
    • extract lat/long from polyline
    • lat/long normalized using pre-calculated mean and std
    • origin_call and taxi_id converted to continuous numbers (0 to ...) to get embeddings later
    • categories convert to cat codes
    
    """
    
    # origin_call -- train
    train.ORIGIN_CALL.fillna(value = -1, inplace=True)
    id_uniq = train.ORIGIN_CALL.unique()
    
    origin_call_dict = {o:i for i,o in enumerate(id_uniq)}

    train['ORIGIN_CALL'] = train.ORIGIN_CALL.apply(lambda x: 0 if x == -1 
                                                   or x == '' else origin_call_dict[x])
    
    
    # origin_call --test (g list of ids in test not in train)
    test.ORIGIN_CALL.fillna(value = -1, inplace=True)

    test['ORIGIN_CALL'] = test.ORIGIN_CALL.apply(lambda x: 0 if x == -1 or x == '' 
                                              else (-2 if x in g[1:] else origin_call_dict[x]))
    
    
    # origin_stand
    train['ORIGIN_STAND']= pd.Series([0 if pd.isnull(x) or x=='' else int(x) for x in train["ORIGIN_STAND"]])
    test['ORIGIN_STAND']= pd.Series([0 if pd.isnull(x) or x=='' else int(x) for x in test["ORIGIN_STAND"]])
    
    # taxi_id
    id_uniq = train.TAXI_ID.unique()
    taxi_id_dict = {o:i for i,o in enumerate(id_uniq)}
    train['TAXI_ID'] = train.TAXI_ID.apply(lambda x: taxi_id_dict[x])
    test['TAXI_ID'] = test.TAXI_ID.apply(lambda x: taxi_id_dict[x])

    # day_type and call_type
    # want 0 for A, 1 for B and 2 for C
    train['DAY_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in train['DAY_TYPE']])
    train['CALL_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in train['CALL_TYPE']])
    test['DAY_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in test['DAY_TYPE']])
    test['CALL_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in test['CALL_TYPE']])
    
    
    # polyline
    polyline1 = pd.Series([ast.literal_eval(x) for x in train['POLYLINE']])
    polyline2 = pd.Series([ast.literal_eval(x) for x in test['POLYLINE']])
    
    # latitude and longitude
    train['LATITUDE'] = pd.Series([np.array([point[1] for point in poly],dtype=np.float32) for poly in polyline1])
    train['LONGITUDE'] = pd.Series([np.array([point[0] for point in poly],dtype=np.float32) for poly in polyline1])
    
    test['LATITUDE'] = pd.Series([np.array([point[1] for point in poly],dtype=np.float32) for poly in polyline2])
    test['LONGITUDE'] = pd.Series([np.array([point[0] for point in poly],dtype=np.float32) for poly in polyline2])
    
    # target variable i.e. last coordinates
    train['TARGET'] = pd.Series([[l[1][0][-1], l[1][1][-1]] if len(l[1][0]) > 1 
                                else np.nan for l in train[['LONGITUDE','LATITUDE']].iterrows()])

    # normalized lat long. We need normalized data for neural nets and this is like cont. variable (not cat)
    train['LATITUDE'] = pd.Series([(t-lat_mean)/lat_std for t in train['LATITUDE']])
    test['LATITUDE'] = pd.Series([(t-lat_mean)/lat_std for t in test['LATITUDE']])
    
    train['LONGITUDE'] = pd.Series([(t-long_mean)/long_std for t in train['LONGITUDE']])
    test['LONGITUDE'] = pd.Series([(t-long_mean)/long_std for t in test['LONGITUDE']])

    # first 5 and last 5 features
    train['COORD_FEATURES'], test['COORD_FEATURES'] = start_stop_inputs(5, train, test)
    
    # day of week variable
    train['DAY_OF_WEEK'] = pd.Series([datetime.datetime.fromtimestamp(t).weekday() for t in train['TIMESTAMP']])
    test['DAY_OF_WEEK'] = pd.Series([datetime.datetime.fromtimestamp(t).weekday() for t in test['TIMESTAMP']])

    # hour of day
    train['HOUR'] = pd.Series([datetime.datetime.fromtimestamp(t).hour for t in train['TIMESTAMP']])
    test['HOUR'] = pd.Series([datetime.datetime.fromtimestamp(t).hour for t in test['TIMESTAMP']])

    # week of year
    train['WEEK_OF_YEAR'] = pd.Series([datetime.datetime.fromtimestamp(t).isocalendar()[1] for t in train['TIMESTAMP']])
    test['WEEK_OF_YEAR'] = pd.Series([datetime.datetime.fromtimestamp(t).isocalendar()[1] for t in test['TIMESTAMP']])

        
    #data = data.dropna()

    return train, test

In [77]:
train2, test2 = train.copy(), test.copy()

In [ ]:
train2, test2 = feature_ext(train2, test2)

In [9]:
train2[:2]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE,TARGET,COORD_FEATURES,DAY_OF_WEEK,HOUR,WEEK_OF_YEAR
0,1372636858620000589,2,0,0,0,1372636858,0,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[-0.21451, -0.214974, -0.199688, -0.182087, -0...","[-0.0437321, -0.0412145, -0.0731591, -0.105104...","[-8.63084, 41.1545]","[-0.0437321, -0.0412145, -0.0731591, -0.105104...",0,0,27
1,1372637303620000596,1,0,7,1,1372637303,0,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...","[0.0339161, 0.0345337, 0.0378275, 0.0429227, 0...","[-0.414429, -0.423249, -0.455494, -0.494991, -...","[-8.66574, 41.1707]","[-0.414429, -0.423249, -0.455494, -0.494991, -...",0,0,27


In [8]:
test2[:2]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE,COORD_FEATURES,DAY_OF_WEEK,HOUR,WEEK_OF_YEAR
0,T1,1,0,15,150,1408039037,0,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[...","[-0.118578, -0.116982, -0.1141, -0.113122, -0....","[0.532604, 0.531971, 0.532454, 0.531671, 0.527...","[0.532604, 0.531971, 0.532454, 0.531671, 0.527...",3,17,33
1,T2,1,0,57,306,1408038611,0,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-...","[-0.158413, -0.158258, -0.155736, -0.150024, -...","[0.0920491, 0.0923659, 0.0915823, 0.0996017, 0...","[0.0920491, 0.0923659, 0.0915823, 0.0996017, 0...",3,17,33


Saving transformed files as `pickle`

Saving files

In [101]:
train2.to_pickle(path+'train2pickle')
test2.to_pickle(path+'test2pickle')

Loading train and test. **Can directly run this**

In [6]:
train2 = pd.read_pickle('train2pickle')
test2 = pd.read_pickle('test2pickle')

After running `fit`, I got to know that we can not make predictions for **embeddings = -2** as it is not being trained. So there are only **20** such records, I am changing those as **0** in test set

In [7]:
test2.ORIGIN_CALL = test2.ORIGIN_CALL.apply(lambda x: 0 if x == -2 else x)

### Step 2: Meanshift clustering

** From the paper**

As the destination we aim to predict is composed of two scalar values (latitude and longitude), it is natural to have two output neurons. However, we found that it was difficult to train such a simple model because it does not take into account any prior information on the distribution of the data. To tackle this issue, we integrate prior knowledge of the destinations directly in the architecture of our model: instead of predicting directly the destination position, we use a predefined set (ci)1≤i≤C of a few thousand destination cluster centers and a hidden layer
that associates a scalar value (pi)i (similar to a probability) to each of these clusters. As the network must output a single destination position, for our output prediction yˆ, we compute a weighted average of the predefined destination cluster centers:

In [9]:
import os
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets.samples_generator import make_blobs
from itertools import cycle
import scipy.misc

In [8]:
# checking how to index and subset lat and long (lat is in 1 and long in 0th)
train2.TARGET[:10][1][1], train2.TARGET[:10][1][0]

(41.17067, -8.66574)

We don't have polyline data for all the training data. For those, we have `nan` in `TARGET` variable and those won't contribute in centroid calculation. 

In [40]:
print("Generating arrival point list")
dests = []

# for l in train2[['LONGITUDE','LATITUDE']].iterrows():
#     if len(l[1][0]) == 0 or len(l[1][1]) == 0: continue
#     dests.append((l[1][1][-1],  l[1][0][-1])) # latitude, longitude

target = train2.TARGET.fillna(0)    
for l in target:
    if l == 0 : continue
    dests.append((l[1],  l[0])) # latitude, longitude
    
pts = np.array(dests)

with open(os.path.join(path, "arrivals.pkl"), "wb") as f:
    pickle.dump(pts, f, protocol=pickle.HIGHEST_PROTOCOL)

print("Doing clustering")
bw = estimate_bandwidth(pts, quantile=.1, n_samples=1000)
print(bw)
bw = 0.001 

ms = MeanShift(bandwidth=bw, bin_seeding=True, min_bin_freq=5)
ms.fit(pts)
cluster_centers = ms.cluster_centers_

print("Clusters shape: ", cluster_centers.shape)

with open(os.path.join(path, "arrival-clusters.pkl"), "wb") as f:
    pickle.dump(cluster_centers, f, protocol=pickle.HIGHEST_PROTOCOL)

Generating arrival point list
Doing clustering
0.0215466177411
Clusters shape:  (3422, 2)


I tried above code for --  
1. scaled coordinates. It doesn't give even close number of clusters. got **22644 clusters**. Not quite right. 
2. unscaled coordinates. Got **3422 clusters**. Winner had 3392 clusters. CLOSE and will keep it

### Step 3: Making Validation Split

As given in winner's solution, we are taking validation set as last year's data with same timestamp as of test data timestamps

In [10]:
# Cuts of the test set minus 1 year
cuts = [
    1376503200, # 2013-08-14 18:00
    1380616200, # 2013-10-01 08:30
    1381167900, # 2013-10-07 17:45
    1383364800, # 2013-11-02 04:00
    1387722600  # 2013-12-22 14:30
]

In [11]:
val_indices = []
index = 0
for index, row in train2.iterrows():
    time = row['TIMESTAMP']
    latitude = row['LATITUDE']
    for ts in cuts:
        if time <= ts and time + 15 * (len(latitude) - 1) >= ts:
            val_indices.append(index)
            break
    index += 1

saving and reloading validation indexes

In [23]:
with open(os.path.join(path, "val_idx.pkl"), "wb") as f:
    pickle.dump(val_indices, f)

In [12]:
with open(os.path.join(path, "val_idx.pkl"), "rb") as f:
    val_indices = pickle.load(f)

validation set

For making embedding layers, I will be needing indexes to feed as values. It will be used for doing lookup.

Before making validation set, this is something I am doing after getting stuck with some error in `fit`. I noticed that for `WEEK_OF_YEAR` and for `ORIGIN_STAND`, the cat codes do no match with index. For these 2, I will replace values with indexes. This is an experiment which is unlikely to change anything. Just trying it out -- **NOPE DIDN'T HELP**. Commented below 3 chunks

In [16]:
for c in cat_vars:
    s =0
    for i, v in enumerate(train2[c].unique()):
        s += i != v
    print(s, c)

In [96]:
# def EmbeddingDataPreprocess(data, cats, inplace=True):
#     ### Each categorical column should have indices as values
#     ### Which will be looked up at embedding matrix and used in modeling
#     ### Make changes inplace
#     if inplace:
#         for c in cats:
#             data[c].replace({val: i for i, val in enumerate(data[c].unique())}, inplace=True)
#         return data
#     else:
#         data_copy = data.copy()
#         for c in cats:
#             data_copy[c].replace({val: i for i, val in enumerate(data_copy[c].unique())}, inplace=True)
#         return data_copy

In [97]:
# EmbeddingDataPreprocess(train2, ['ORIGIN_STAND', 'WEEK_OF_YEAR'], inplace=True)

In [98]:
# for c in cat_vars:
#     s =0
#     for i, v in enumerate(train2[c].unique()):
#         s += i != v
#     print(s, c)

**Index == values** now

Found that in below codes, index of embedding for `WEEK_OF_YEAR` starts from 1 and others from 0. I am going to change that to make it start from 0 because I guess it is the reason of `cude runtime error` that I am getting. Experiment!!


In [13]:
# making week of year from 0-51 from 1-52

train2.WEEK_OF_YEAR = train2.WEEK_OF_YEAR - 1

In [14]:
# transformation on test set
test2.WEEK_OF_YEAR = test2.WEEK_OF_YEAR - 1

In [15]:
X_valid = train2.iloc[val_indices]

Let's check if we have right timestamps in validation set or not

In [16]:
lt = []
for d in X_valid['TIMESTAMP']:
    lt.append(datetime.date.fromtimestamp(d))
    
set(lt)

{datetime.date(2013, 8, 14),
 datetime.date(2013, 10, 1),
 datetime.date(2013, 10, 7),
 datetime.date(2013, 11, 2),
 datetime.date(2013, 12, 22)}

Yup. above dates match with our dates of test set. Just 1 previous year

In [17]:
X_train = train2.drop(train2.index[[val_indices]])

In [18]:
len(X_valid), len(X_train)

(304, 1710366)

Validation set length is similar as test set length

In [19]:
with open(os.path.join(path, "arrival-clusters.pkl"), "rb") as f:
    cluster_centers2 = pickle.load(f)

In [20]:
len(cluster_centers2)

3422

Saving `X_train` and `X_valid`

In [21]:
X_train.to_pickle(path + 'X_train.pkl')
X_valid.to_pickle(path + 'X_valid.pkl')

Reading validation and training files. Next time can directly read from here

In [184]:
X_train = pd.read_pickle('X_train.pkl')
X_valid = pd.read_pickle('X_valid.pkl')

### Step 4: DL

Hidden layer is basically telling probability of observation being in ith cluster. So it will have as many outputs as number of clusters. (3422). It includes training for `probabilities`  

Now sum of all the `probabilities` should be 1. Therefore we will use `softmax` over a trained linear layer


On these 3422 output from hidden, we take weighted avg which will just be sum of multiplied probability into cluster center (i.e. sum of 3422 elements). It includes no traning, just using previous probabilities, multiplying by fixed crntroid coordinated and calculating weighted avg.   

Think of this operation as a linear layer with `weights = centers (i.e. fixed) and input = probabilities`



Layers to add:  
1. Pass input data to a **dense layer** which has 500 outputs and those **500 to relu**. 
1. Pass coming data to a **linear layer** which has 3422 outputs
2. These 3422 outputs passed to a **softmax** (which will give p)
3. These 3422 p's will go to **linear layer** which **does not** update weights.  just multiply p and c . 
4. 


#### Analysing cardinality for embeddings

In [22]:
cols = ['ORIGIN_CALL', 'TAXI_ID', 'ORIGIN_STAND', 'HOUR', 'DAY_OF_WEEK', 'WEEK_OF_YEAR']

In [23]:
for i in cols:
    print(i, '--', 'Possible values', len(train2[i].unique()) ,'||', 'Embedding size', 10)

ORIGIN_CALL -- Possible values 57106 || Embedding size 10
TAXI_ID -- Possible values 448 || Embedding size 10
ORIGIN_STAND -- Possible values 64 || Embedding size 10
HOUR -- Possible values 24 || Embedding size 10
DAY_OF_WEEK -- Possible values 7 || Embedding size 10
WEEK_OF_YEAR -- Possible values 52 || Embedding size 10


They took embedding of size 10 for all the above features

In [24]:
emb_szs = []
for i in cols:
    emb_szs.append((len(train2[i].unique()), 10))
emb_szs

[(57106, 10), (448, 10), (64, 10), (24, 10), (7, 10), (52, 10)]

In [25]:
cat_vars = ['ORIGIN_CALL', 'TAXI_ID', 'ORIGIN_STAND', 'HOUR', 'DAY_OF_WEEK', 'WEEK_OF_YEAR']
#cont_vars = 

#### Getting continuous features

In [26]:
X_train[:1]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE,TARGET,COORD_FEATURES,DAY_OF_WEEK,HOUR,WEEK_OF_YEAR
0,1372636858620000589,2,0,0,0,1372636858,0,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[-0.21451, -0.214974, -0.199688, -0.182087, -0...","[-0.0437321, -0.0412145, -0.0731591, -0.105104...","[-8.63084, 41.1545]","[-0.0437321, -0.0412145, -0.0731591, -0.105104...",0,0,26


In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
# replacing NA with 0s
# k = 5

X_train.COORD_FEATURES.fillna(0, inplace = True)
X_valid.COORD_FEATURES.fillna(0, inplace = True)

In [28]:
test2.COORD_FEATURES.fillna(0, inplace = True)

In [29]:
def get_features(df, target = True):
    
    cols = ['ORIGIN_CALL', 'TAXI_ID', 'ORIGIN_STAND', 'HOUR', 'DAY_OF_WEEK', 'WEEK_OF_YEAR','MISSING_DATA']
    
    df_feat = df[cols]
    for i in range(10):
        df_feat[f'long_{i}'] = df.COORD_FEATURES.apply(lambda x: 0 if type(x) == int else x[i]) # creating 10 cols for long
        df_feat[f'lat_{i}'] = df.COORD_FEATURES.apply(lambda x: 0 if type(x) == int else x[i+10]) # 10 cols for lat

    if target is True:
        df_target = df['TARGET']
    else:
        df_target = None
               
    return df_feat, df_target

Training and validation sets

In [30]:
train2.COORD_FEATURES.fillna(0, inplace = True)
df, y = get_features(train2)

In [31]:
train_feat, train_target = get_features(X_train)

In [32]:
valid_feat, valid_target = get_features(X_valid) 

In [33]:
test_feat, test_target = get_features(test2, target = False)

Centers of cluster lats and longs. Needed for last nn layer

In [34]:
cluster_lats = []

for i in cluster_centers2:
    cluster_lats.append(i[0])
    
cluster_longs = []

for i in cluster_centers2:
    cluster_longs.append(i[1])

In [35]:
cluster_lats = np.array(cluster_lats)[:,None]
cluster_longs = np.array(cluster_longs)[:,None]

Mixed input nn model

#### Data loader

Customizing `fast.ai` functions

#### After loading data into dataloader, writing my own training loop (tried `get_learner` from fastai. doesn't work for me)

In [36]:
class taxi_dataset_from_df():
    def __init__(self, df, cats, y):
        #n = len(cats[0]) if cats else len(conts[0])
        
        self.df_cats = df[cats]
        self.df_conts = df.drop(cats, axis=1)
        
        lat = y.fillna(0).apply(lambda x: 0 if type(x) == int else x[1])
        long = y.fillna(0).apply(lambda x: 0 if type(x) == int else x[0])
        
        target = pd.DataFrame({'long': long.astype(np.float32),'lat' : lat.astype(np.float32)})
        
        cat_cols = [c.values for n,c in self.df_cats.items()]
        cont_cols = [c.values for n,c in self.df_conts.items()]  
        target_cols = [c.values for n,c in target.items()]  
        
        self.cats = np.stack(cat_cols, 1).astype(np.int64)
        self.conts = np.stack(cont_cols, 1).astype(np.float32) 
        self.target = np.stack(target_cols, 1).astype(np.float32) 

    def __len__(self): return len(self.df_cats)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx], self.target[idx]]


In [37]:
train_dataset = taxi_dataset_from_df(train_feat, cat_vars, train_target)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)

In [38]:
valid_feat.shape

(304, 27)

In [39]:
valid_dataset = taxi_dataset_from_df(valid_feat, cat_vars, valid_target)

valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=304, shuffle=False)

In [40]:
class taxi_dataset_from_df_test():
    def __init__(self, df, cats):
        #n = len(cats[0]) if cats else len(conts[0])
        
        self.df_cats = df[cats]
        self.df_conts = df.drop(cats, axis=1)
        
        #lat = y.fillna(0).apply(lambda x: 0 if type(x) == int else x[1])
        #long = y.fillna(0).apply(lambda x: 0 if type(x) == int else x[0])
        
        #target = pd.DataFrame({'long': long.astype(np.float32),'lat' : lat.astype(np.float32)})
        
        cat_cols = [c.values for n,c in self.df_cats.items()]
        cont_cols = [c.values for n,c in self.df_conts.items()]  
        #target_cols = [c.values for n,c in target.items()]  
        
        self.cats = np.stack(cat_cols, 1).astype(np.int64)
        self.conts = np.stack(cont_cols, 1).astype(np.float32) 
       # self.target = np.stack(target_cols, 1).astype(np.float32) 

    def __len__(self): return len(self.df_cats)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx]]


In [41]:
test2.shape

(320, 15)

In [42]:
test_dataset = taxi_dataset_from_df_test(test_feat, cat_vars)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=320, shuffle=False)

let's see what my dataset is returning. It is a list with 3 elements. cat df, cont df and y.

**NOTICE that lat is in 0 here and long is in 1**

In [105]:
train_dataset[0:3]

[array([[ 0,  0,  0,  0,  0, 26],
        [ 0,  1,  7,  0,  0, 26],
        [ 0,  2,  0,  0,  0, 26]]),
 array([[ 0.     , -0.04373, -0.21451, -0.04121, -0.21497, -0.07316, -0.19969, -0.1051 , -0.18209, -0.13656,
         -0.17452, -0.26228, -0.03808, -0.25677, -0.03783, -0.25677, -0.03772, -0.25677, -0.03793, -0.25694,
         -0.03808],
        [ 0.     , -0.41443,  0.03392, -0.42325,  0.03453, -0.45549,  0.03783, -0.49499,  0.04292, -0.53811,
          0.0491 , -0.93476,  0.14369, -0.90834,  0.15754, -0.88284,  0.17128, -0.86757,  0.17977, -0.86711,
          0.18023],
        [ 0.     ,  0.05555, -0.22872,  0.04832, -0.22882,  0.03368, -0.2298 ,  0.02393, -0.22856,  0.0041 ,
         -0.22748, -0.13893, -0.17334, -0.09062, -0.18765, -0.05774, -0.2071 , -0.02139, -0.22192,  0.003  ,
         -0.2264 ]], dtype=float32),
 array([[ 41.15449,  -8.63084],
        [ 41.17067,  -8.66574],
        [ 41.14053,  -8.61597]], dtype=float32)]

In [106]:
train_dataset[:][0]

array([[  0,   0,   0,   0,   0,  26],
       [  0,   1,   7,   0,   0,  26],
       [  0,   2,   0,   0,   0,  26],
       ..., 
       [  0, 439,   0,  10,   4,   0],
       [  0, 443,  12,  15,   0,  26],
       [  0, 444,  34,  19,   0,  26]])

In [43]:
np.min(test_dataset[:][0], 0), np.max(test_dataset[:][0], 0)

(array([ 0,  0,  0,  2,  0, 32]),
 array([47093,   444,    63,    17,     6,    50]))

In [44]:
np.min(train_dataset[:][0], 0), np.max(train_dataset[:][0], 0)

(array([0, 0, 0, 0, 0, 0]), array([57105,   447,    63,    23,     6,    51]))

In [45]:
np.min(valid_dataset[:][0], 0), np.max(valid_dataset[:][0], 0)

(array([ 0,  0,  0,  3,  0, 32]),
 array([34587,   435,    63,    17,     6,    50]))

Ok. So let's think on how will I use input of these for in dataloader and that into my `Net`

#### Defining Loss function

They used `equirectangular distance` as loss function which performed better than original kaggle metric (`Haversine distance`). Actually equirectangular distance is just a simpler approximation to haversine distance

Will keep long in 0 and lat in 1 as there in target column

In [194]:
# a predicted, b true
# data loader is in such a way that true y should be indexed from floatTensor

class equiRectLoss(nn.Module):

    def __init__(self):
        super(equiRectLoss, self).__init__()
        self.rearth = 6371
        self.deg2rad = 3.141592653589793 / 180.

    def forward(self, a,b):  
        lat1 = a[:,0]
        lon1 = a[:,1]
        lat2 = b[:,0]
        lon2 = b[:,1]   
        
        x = (lon2-lon1) * torch.cos((lat1+lat2)/2)
        y = (lat2-lat1)

        loss = torch.mean((torch.sqrt(x**2 + y**2) * self.rearth),dim=0)
        return loss

#### defining nn

In [46]:
cluster_latsV = torch.Tensor(cluster_lats)
cluster_longsV = torch.Tensor(cluster_longs)
clus_latlong = torch.cat((cluster_latsV, cluster_longsV), dim=1)

In [47]:
clus_latlong.shape

torch.Size([3422, 2])

In [48]:
def emb_init(x):
    x = x.weight.data
    sc = 2/(x.size(1)+1)
    x.uniform_(-sc,sc)

In [49]:
def final_layer_wt(x):
    x = x.weight.data
    x = clus_latlong
    return x

In [50]:
# they didn't do batchnormm in paper            
class Net(nn.Module):
    def __init__(self, emb_szs, n_cont,
                 y_range=None, use_bn=False):
        super().__init__()
        
        self.embs = nn.ModuleList([nn.Embedding(c, s) for c,s in emb_szs])
        for emb in self.embs: emb_init(emb)
        #n_emb = sum(e.embedding_dim for e in self.embs)
        n_emb = 60
        
        # 6 cat vars . therefore 60 embeddings (10 each)
        self.lin1 = nn.Linear(n_emb + n_cont, 500) # 1st linear layer -- no. features x 500
        self.relu = nn.ReLU()   # 2nd relu layer (output = 500 neurons)
        self.lin2 = nn.Linear(500, 3422)  # 3rd linear layer. 500x3422
        self.soft = nn.Softmax() # 4th softmax layer. 3422 probabilities (will sum to 1)
         
        for o in [self.lin1, self.lin2] : kaiming_normal(o.weight.data) #weight initialization for lin layers
            
        self.bn = nn.BatchNorm1d(n_cont) 
        
        self.lin3 = nn.Linear(3422,2)
        
        for o in [self.lin3]: final_layer_wt(o)
        #for o in [self.lin3] : kaiming_normal(o.weight.data)

    def forward(self, x_cat, x_cont):
        x = [e(x_cat[:,i]) for i,e in enumerate(self.embs)]
        x = torch.cat(x, 1)
        x2 = self.bn(x_cont)
        x = torch.cat([x, x2], 1)
        
        out = self.lin1(x)
        out = self.relu(out)
        out = self.lin2(out)
        out = self.soft(out)
#         outlat = out.mm(cluster_latsV)  # out is n*3422 and cluster_lats is 3422*1
#         outlong = out.mm(cluster_longsV)
        out = self.lin3(out)
        
        #target = pd.DataFrame({'long': outlong.astype(np.float32),'lat' : outlat.astype(np.float32)})
        #target_cols = [c.values for n,c in target.items()]  
        #target_out = torch.cat((cluster_latsV, cluster_longsV), dim=1)

        return out # nx2 tensor where n is no. of observations

In [51]:
it = iter(train_loader)
a,b,c = next(it)

In [52]:
a.shape, b.shape, c.shape

(torch.Size([200, 6]), torch.Size([200, 21]), torch.Size([200, 2]))

In [53]:
def score(x, y):
    y_pred = to_np(net2(V(x)))
    return np.sum(y_pred.argmax(axis=1) == to_np(y))/len(y_pred)

In [55]:
net1 = Net(emb_szs= emb_szs, n_cont= len(df.columns)-len(cat_vars), y_range=y)
net1 = net1.cuda()
loss= F.mse_loss
learning_rate = 1e-2 # can try different values, cyclic and diff learning rates
#optimizer=optim.SGD(net2.parameters(), lr=learning_rate, momentum = 0.9)
optimizer=optim.Adam(net1.parameters(), lr=learning_rate) # will check if weight_decay needed or not later


for epoch in range(2):
    losses=[]
    losses_val = []
    running_loss = 0.0
    #running_loss_valid = 0.0
    examples = 0
    #dl = iter(train_loader)
    for i, data in enumerate(train_loader, 0):
    #for t in range(len(dl)):
        # Forward pass: compute predicted y and loss by passing x to the model.
        xcat, xcont, yt = data      
        y_pred = net1(V(xcat), V(xcont))
        l = loss(y_pred, V(yt))
        
#         xcat_val, xcont_val, yt_val = valid_dataset[:]
#         l_valid = loss(net1(V(T(xcat_val)), V(T(xcont_val))), V(T(yt_val)))
        
#         losses_val.append(l_valid)
        
        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable weights of the model)
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model parameters
        l.backward()

        # Calling the step function on an Optimizer makes an update to its parameters
        optimizer.step()
        
        # Print statistics
        running_loss += l.data[0]
        #running_loss_valid += l_valid.data[0]
        examples += 10
        
        print('[%d, %5d] Training loss: %.3f' % (epoch + 1, i + 1, running_loss / examples))
        
        #print('[%d, %5d] Training loss: %.3f Validation loss: %.3f' % (epoch + 1, i + 1, running_loss / examples, 
                                                 # running_loss_valid/examples))
        
print('Finished Training')

[1,     1] Training loss: 87.158
[1,     2] Training loss: 85.766
[1,     3] Training loss: 85.712
[1,     4] Training loss: 85.437
[1,     5] Training loss: 85.691
[1,     6] Training loss: 85.552
[1,     7] Training loss: 85.500
[1,     8] Training loss: 85.557
[1,     9] Training loss: 85.496
[1,    10] Training loss: 85.699
[1,    11] Training loss: 85.698
[1,    12] Training loss: 85.763
[1,    13] Training loss: 85.676
[1,    14] Training loss: 85.594
[1,    15] Training loss: 85.575
[1,    16] Training loss: 85.553
[1,    17] Training loss: 85.502
[1,    18] Training loss: 85.475
[1,    19] Training loss: 85.446
[1,    20] Training loss: 85.437
[1,    21] Training loss: 85.342
[1,    22] Training loss: 85.350
[1,    23] Training loss: 85.222
[1,    24] Training loss: 85.225
[1,    25] Training loss: 85.191
[1,    26] Training loss: 85.122
[1,    27] Training loss: 85.054
[1,    28] Training loss: 84.988
[1,    29] Training loss: 84.953
[1,    30] Training loss: 84.931
[1,    31]

[1,   253] Training loss: 74.489
[1,   254] Training loss: 74.441
[1,   255] Training loss: 74.400
[1,   256] Training loss: 74.360
[1,   257] Training loss: 74.323
[1,   258] Training loss: 74.282
[1,   259] Training loss: 74.241
[1,   260] Training loss: 74.197
[1,   261] Training loss: 74.156
[1,   262] Training loss: 74.113
[1,   263] Training loss: 74.072
[1,   264] Training loss: 74.033
[1,   265] Training loss: 73.991
[1,   266] Training loss: 73.948
[1,   267] Training loss: 73.909
[1,   268] Training loss: 73.870
[1,   269] Training loss: 73.825
[1,   270] Training loss: 73.784
[1,   271] Training loss: 73.744
[1,   272] Training loss: 73.699
[1,   273] Training loss: 73.659
[1,   274] Training loss: 73.620
[1,   275] Training loss: 73.581
[1,   276] Training loss: 73.538
[1,   277] Training loss: 73.496
[1,   278] Training loss: 73.459
[1,   279] Training loss: 73.421
[1,   280] Training loss: 73.382
[1,   281] Training loss: 73.339
[1,   282] Training loss: 73.297
[1,   283]

[1,   517] Training loss: 64.683
[1,   518] Training loss: 64.649
[1,   519] Training loss: 64.616
[1,   520] Training loss: 64.582
[1,   521] Training loss: 64.550
[1,   522] Training loss: 64.516
[1,   523] Training loss: 64.483
[1,   524] Training loss: 64.450
[1,   525] Training loss: 64.417
[1,   526] Training loss: 64.383
[1,   527] Training loss: 64.349
[1,   528] Training loss: 64.314
[1,   529] Training loss: 64.281
[1,   530] Training loss: 64.249
[1,   531] Training loss: 64.216
[1,   532] Training loss: 64.184
[1,   533] Training loss: 64.150
[1,   534] Training loss: 64.117
[1,   535] Training loss: 64.084
[1,   536] Training loss: 64.051
[1,   537] Training loss: 64.017
[1,   538] Training loss: 63.984
[1,   539] Training loss: 63.950
[1,   540] Training loss: 63.917
[1,   541] Training loss: 63.885
[1,   542] Training loss: 63.852
[1,   543] Training loss: 63.818
[1,   544] Training loss: 63.785
[1,   545] Training loss: 63.753
[1,   546] Training loss: 63.720
[1,   547]

[1,   781] Training loss: 56.620
[1,   782] Training loss: 56.592
[1,   783] Training loss: 56.564
[1,   784] Training loss: 56.536
[1,   785] Training loss: 56.508
[1,   786] Training loss: 56.480
[1,   787] Training loss: 56.452
[1,   788] Training loss: 56.425
[1,   789] Training loss: 56.397
[1,   790] Training loss: 56.370
[1,   791] Training loss: 56.342
[1,   792] Training loss: 56.315
[1,   793] Training loss: 56.287
[1,   794] Training loss: 56.259
[1,   795] Training loss: 56.231
[1,   796] Training loss: 56.204
[1,   797] Training loss: 56.176
[1,   798] Training loss: 56.148
[1,   799] Training loss: 56.120
[1,   800] Training loss: 56.093
[1,   801] Training loss: 56.066
[1,   802] Training loss: 56.038
[1,   803] Training loss: 56.011
[1,   804] Training loss: 55.983
[1,   805] Training loss: 55.955
[1,   806] Training loss: 55.927
[1,   807] Training loss: 55.900
[1,   808] Training loss: 55.872
[1,   809] Training loss: 55.845
[1,   810] Training loss: 55.817
[1,   811]

[1,  1045] Training loss: 49.826
[1,  1046] Training loss: 49.802
[1,  1047] Training loss: 49.778
[1,  1048] Training loss: 49.754
[1,  1049] Training loss: 49.731
[1,  1050] Training loss: 49.707
[1,  1051] Training loss: 49.683
[1,  1052] Training loss: 49.660
[1,  1053] Training loss: 49.636
[1,  1054] Training loss: 49.613
[1,  1055] Training loss: 49.589
[1,  1056] Training loss: 49.566
[1,  1057] Training loss: 49.542
[1,  1058] Training loss: 49.519
[1,  1059] Training loss: 49.495
[1,  1060] Training loss: 49.472
[1,  1061] Training loss: 49.448
[1,  1062] Training loss: 49.425
[1,  1063] Training loss: 49.401
[1,  1064] Training loss: 49.378
[1,  1065] Training loss: 49.354
[1,  1066] Training loss: 49.331
[1,  1067] Training loss: 49.308
[1,  1068] Training loss: 49.284
[1,  1069] Training loss: 49.261
[1,  1070] Training loss: 49.238
[1,  1071] Training loss: 49.214
[1,  1072] Training loss: 49.191
[1,  1073] Training loss: 49.168
[1,  1074] Training loss: 49.144
[1,  1075]

...

[2,  7271] Training loss: 1.842
[2,  7272] Training loss: 1.842
[2,  7273] Training loss: 1.842
[2,  7274] Training loss: 1.842
[2,  7275] Training loss: 1.842
[2,  7276] Training loss: 1.843
[2,  7277] Training loss: 1.843
[2,  7278] Training loss: 1.843
[2,  7279] Training loss: 1.843
[2,  7280] Training loss: 1.843
[2,  7281] Training loss: 1.843
[2,  7282] Training loss: 1.842
[2,  7283] Training loss: 1.843
[2,  7284] Training loss: 1.842
[2,  7285] Training loss: 1.842
[2,  7286] Training loss: 1.842
[2,  7287] Training loss: 1.842
[2,  7288] Training loss: 1.842
[2,  7289] Training loss: 1.843
[2,  7290] Training loss: 1.843
[2,  7291] Training loss: 1.843
[2,  7292] Training loss: 1.843
[2,  7293] Training loss: 1.843
[2,  7294] Training loss: 1.843
[2,  7295] Training loss: 1.843
[2,  7296] Training loss: 1.843
[2,  7297] Training loss: 1.843
[2,  7298] Training loss: 1.843
[2,  7299] Training loss: 1.843
[2,  7300] Training loss: 1.843
[2,  7301] Training loss: 1.844
[2,  730

[2,  7530] Training loss: 1.844
[2,  7531] Training loss: 1.844
[2,  7532] Training loss: 1.844
[2,  7533] Training loss: 1.844
[2,  7534] Training loss: 1.844
[2,  7535] Training loss: 1.844
[2,  7536] Training loss: 1.845
[2,  7537] Training loss: 1.844
[2,  7538] Training loss: 1.844
[2,  7539] Training loss: 1.844
[2,  7540] Training loss: 1.845
[2,  7541] Training loss: 1.844
[2,  7542] Training loss: 1.844
[2,  7543] Training loss: 1.845
[2,  7544] Training loss: 1.845
[2,  7545] Training loss: 1.844
[2,  7546] Training loss: 1.845
[2,  7547] Training loss: 1.845
[2,  7548] Training loss: 1.844
[2,  7549] Training loss: 1.844
[2,  7550] Training loss: 1.844
[2,  7551] Training loss: 1.844
[2,  7552] Training loss: 1.844
[2,  7553] Training loss: 1.844
[2,  7554] Training loss: 1.844
[2,  7555] Training loss: 1.844
[2,  7556] Training loss: 1.844
[2,  7557] Training loss: 1.844
[2,  7558] Training loss: 1.844
[2,  7559] Training loss: 1.844
[2,  7560] Training loss: 1.844
[2,  756

[2,  7794] Training loss: 1.846
[2,  7795] Training loss: 1.846
[2,  7796] Training loss: 1.846
[2,  7797] Training loss: 1.846
[2,  7798] Training loss: 1.845
[2,  7799] Training loss: 1.845
[2,  7800] Training loss: 1.846
[2,  7801] Training loss: 1.846
[2,  7802] Training loss: 1.846
[2,  7803] Training loss: 1.846
[2,  7804] Training loss: 1.846
[2,  7805] Training loss: 1.846
[2,  7806] Training loss: 1.846
[2,  7807] Training loss: 1.846
[2,  7808] Training loss: 1.846
[2,  7809] Training loss: 1.846
[2,  7810] Training loss: 1.846
[2,  7811] Training loss: 1.846
[2,  7812] Training loss: 1.846
[2,  7813] Training loss: 1.846
[2,  7814] Training loss: 1.846
[2,  7815] Training loss: 1.846
[2,  7816] Training loss: 1.846
[2,  7817] Training loss: 1.846
[2,  7818] Training loss: 1.846
[2,  7819] Training loss: 1.846
[2,  7820] Training loss: 1.846
[2,  7821] Training loss: 1.846
[2,  7822] Training loss: 1.846
[2,  7823] Training loss: 1.846
[2,  7824] Training loss: 1.846
[2,  782

[2,  8058] Training loss: 1.847
[2,  8059] Training loss: 1.847
[2,  8060] Training loss: 1.847
[2,  8061] Training loss: 1.847
[2,  8062] Training loss: 1.847
[2,  8063] Training loss: 1.847
[2,  8064] Training loss: 1.847
[2,  8065] Training loss: 1.847
[2,  8066] Training loss: 1.847
[2,  8067] Training loss: 1.847
[2,  8068] Training loss: 1.847
[2,  8069] Training loss: 1.847
[2,  8070] Training loss: 1.847
[2,  8071] Training loss: 1.847
[2,  8072] Training loss: 1.847
[2,  8073] Training loss: 1.847
[2,  8074] Training loss: 1.847
[2,  8075] Training loss: 1.847
[2,  8076] Training loss: 1.848
[2,  8077] Training loss: 1.848
[2,  8078] Training loss: 1.848
[2,  8079] Training loss: 1.848
[2,  8080] Training loss: 1.847
[2,  8081] Training loss: 1.848
[2,  8082] Training loss: 1.848
[2,  8083] Training loss: 1.848
[2,  8084] Training loss: 1.848
[2,  8085] Training loss: 1.847
[2,  8086] Training loss: 1.847
[2,  8087] Training loss: 1.847
[2,  8088] Training loss: 1.847
[2,  808

[2,  8322] Training loss: 1.847
[2,  8323] Training loss: 1.847
[2,  8324] Training loss: 1.847
[2,  8325] Training loss: 1.847
[2,  8326] Training loss: 1.847
[2,  8327] Training loss: 1.847
[2,  8328] Training loss: 1.847
[2,  8329] Training loss: 1.847
[2,  8330] Training loss: 1.847
[2,  8331] Training loss: 1.847
[2,  8332] Training loss: 1.847
[2,  8333] Training loss: 1.847
[2,  8334] Training loss: 1.847
[2,  8335] Training loss: 1.847
[2,  8336] Training loss: 1.847
[2,  8337] Training loss: 1.846
[2,  8338] Training loss: 1.846
[2,  8339] Training loss: 1.847
[2,  8340] Training loss: 1.847
[2,  8341] Training loss: 1.846
[2,  8342] Training loss: 1.846
[2,  8343] Training loss: 1.846
[2,  8344] Training loss: 1.846
[2,  8345] Training loss: 1.846
[2,  8346] Training loss: 1.846
[2,  8347] Training loss: 1.846
[2,  8348] Training loss: 1.846
[2,  8349] Training loss: 1.846
[2,  8350] Training loss: 1.846
[2,  8351] Training loss: 1.846
[2,  8352] Training loss: 1.846
[2,  835

In [ ]:
# def print_stats(epoch, names, values):
#     print_str = "{}: {:3d}, ".format("epoch",epoch)
#     print_str += ', '.join(("{}: {:.6f}".format(n, v) for (n, v) in zip(names, values)))
#     print(print_str)  

In [56]:
net1.parameters

<bound method Module.parameters of Net (
  (embs): ModuleList (
    (0): Embedding(57106, 10)
    (1): Embedding(448, 10)
    (2): Embedding(64, 10)
    (3): Embedding(24, 10)
    (4): Embedding(7, 10)
    (5): Embedding(52, 10)
  )
  (lin1): Linear (81 -> 500)
  (relu): ReLU ()
  (lin2): Linear (500 -> 3422)
  (soft): Softmax ()
  (bn): BatchNorm1d(21, eps=1e-05, momentum=0.1, affine=True)
  (lin3): Linear (3422 -> 2)
)>

In [238]:
# for i, data in enumerate(test_loader, 0):
#     xcat_test, xcont_test = data      
#     test_pred = net1(V(xcat_test), V(xcont_test))

In [60]:
xcat_test, xcont_test = test_dataset[:]
test_pred = net1(V(T(xcat_test)), V(T(xcont_test)))

In [61]:
test_pred

Variable containing:
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.1525  -8.3958
 40.152